In [ ]:
# Exemplo do mestre Fernando Masanori (@fmasanori) para quebrar captchas
# Uso do serviço http://api.dbcapi.me/api

In [1]:
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup
import subprocess
import requests


In [2]:
html = urlopen("http://www.pythonscraping.com/humans-only")
bsObj = BeautifulSoup(html, "html.parser")

In [3]:
#Gather prepopulated form values
imageLocation = bsObj.find("img", {"title": "Image CAPTCHA"})["src"]
formBuildId = bsObj.find("input", {"name":"form_build_id"})["value"]
captchaSid = bsObj.find("input", {"name":"captcha_sid"})["value"]
captchaToken = bsObj.find("input", {"name":"captcha_token"})["value"]

In [4]:
captchaUrl = "http://pythonscraping.com"+imageLocation
urlretrieve(captchaUrl, "captcha.jpg")

('captcha.jpg', <http.client.HTTPMessage at 0x7f84d0aa7898>)

In [5]:
def verf_captcha(texto_id):
    while True:
        url_zika = "http://api.dbcapi.me/api/captcha/%s" % texto_id
        get_solved = requests.get(url_zika,timeout=5)
        texto_solved = get_solved.text
        texto_solved = texto_solved.split('&')[2].split('=')[1]
        if len(texto_solved) > 0:
            return texto_solved
        else:
            pass

In [ ]:
# Dados de acesso

In [6]:
data = {"username":"","password":""}
arq = {"captchafile":open("captcha.jpg","rb")}

In [7]:
req_captcha = requests.post("http://api.dbcapi.me/api/captcha", files=arq, data=data,timeout=5)

In [8]:
texto_id = req_captcha.text
texto_id = texto_id.split('&')[1].split('=')[1]
print (texto_id)

125157575


In [9]:
texto_solved = verf_captcha(texto_id)
print (texto_solved)

XNC8P


In [10]:
captchaResponse = texto_solved
print("Captcha solution attempt: "+captchaResponse)

Captcha solution attempt: XNC8P


In [11]:
if len(captchaResponse) == 5:
    params = {"captcha_token":captchaToken, "captcha_sid":captchaSid,   
              "form_id":"comment_node_page_form", "form_build_id": formBuildId, 
                  "captcha_response":captchaResponse, "name":"Ryan Mitchell", 
                  "subject": "I come to seek the Grail", 
                  "comment_body[und][0][value]": 
                                           "...and I am definitely not a bot"}
    r = requests.post("http://www.pythonscraping.com/comment/reply/10", 
                          data=params)
    responseObj = BeautifulSoup(r.text, 'html.parser')
    if responseObj.find("div", {"class":"messages"}) is not None:
        res = responseObj.find("div", {"class":"messages"}).get_text()
        print (res)
        if 'Error' in res:
            req_err = requests.post("http://api.dbcapi.me/api/captcha/%s" %texto_id,
                                        data=data,timeout=5)
else:
    print("There was a problem reading the CAPTCHA correctly!")


Status message
Your comment has been queued for review by site administrators and will be published after approval.
